# Router Continued

## Configuration File Documentation

This configuration file is used by the FMNC Router Application to determine logging behavior, packet handling policies, timing parameters, and SSL management. By default, the application looks for this file at `../config.cfg` relative to the build or executable directory.

## Current Configuration Parameters

### `log_level`
- **Type:** String
- **Default:** `"DEBUG"`
- **Description:**  
  Specifies the verbosity level for logging. Possible values might include:
  - `"TRACE_L3"`, `"TRACE_L2"`, `"TRACE_L1"`: Increasingly verbose trace levels for deep debugging.
  - `"DEBUG"`: Log detailed diagnostic information useful during development.
  - `"INFO"`: Standard operational messages that highlight the flow through the system.
  - `"WARN"`: Potentially harmful situations that require attention.
  - `"ERROR"`: Error events that might still allow the application to continue running.
  - `"CRITICAL"`: Severe error events that likely lead to application termination.

### `log_directory`
- **Type:** String
- **Default:** `""` (empty string)
- **Description:**  
  Defines the directory where log files should be stored. If empty, the application may log to the current working directory or use a default location. Ensure that the directory exists and that the application has permission to write to it.

### `time_wait_ms`
- **Type:** Integer
- **Default:** `1`
- **Description:**  
  Determines the time (in milliseconds) that the application will wait before forwarding packets. A small value (e.g., 1 ms) can simulate near-real-time forwarding, while a larger value (e.g., 100 ms or more) might batch packets and potentially reorder them more noticeably.

### `reorder_packets`
- **Type:** Boolean
- **Default:** `true`
- **Description:**  
  Indicates whether the application should reorder packets before sending them out. When `true`, packets might be reversed or shuffled to simulate out-of-order delivery conditions, useful for testing network resilience or debugging protocol implementations.

### `manage_ssl`
- **Type:** Boolean
- **Default:** `true`
- **Description:**  
  Specifies whether SSL (TLS) connections should be specially handled. When `true`, the application may avoid splicing or modifying SSL payload data, ensuring encrypted sessions remain intact and not corrupted by packet manipulation.

## Additional Configuration Parameters

The following parameters are not currently in the provided config file but could be helpful to add to fine-tune the application's behavior:

### `interface`
- **Type:** String
- **Description:**  
  Network interface name to capture and forward packets on. For example: `"eth0"` or `"wlan0"`. This would simplify configuration instead of hardcoding interfaces in the code.

### `capture_filter`
- **Type:** String
- **Description:**  
  A `libpcap` filter expression that can be applied directly without recomputing in the code. Example: `"tcp and port 80"`.

### `max_payload_size`
- **Type:** Integer
- **Default:** `1000000` (1 MB)
- **Description:**  
  The maximum size of a payload to be processed and potentially split into multiple packets. Adjusting this can help test scenarios with large data transfers.

### `window_size`
- **Type:** Integer
- **Description:**  
  The threshold for accumulated payload data before triggering a "window" mechanism that splits and reorders the payload into multiple packets. Useful for testing congestion-control or window-based packet sending logic.

### `min_payload_size`
- **Type:** Integer
- **Description:**  
  The minimum size for splitting payload chunks. This ensures that each chunk is of a certain minimum length, preventing overly small packets from being generated.

### `delete_old_logs_on_startup`
- **Type:** Boolean
- **Default:** `false`
- **Description:**  
  If set to `true`, the application will delete old JSON log files (`packet_log.json`, `tcp_flag_counts.json`, `tcp_option_counts.json`) on startup to start fresh.

### `serialization_interval_sec`
- **Type:** Integer
- **Default:** `5`
- **Description:**  
  How frequently (in seconds) the serialization manager should write TCP flag counts, TCP option counts, and packet logs to JSON files.

---

## Example of a Configuration File

```cfg
log_level = "DEBUG";
log_directory = "/var/log/fmnc";
time_wait_ms = 1;
reorder_packets = true;
manage_ssl = true;

interface = "eth0";
capture_filter = "src host 192.168.2.2 and tcp";
max_payload_size = 2000000;  # 2 MB
window_size = 65535;
min_payload_size = 512;
delete_old_logs_on_startup = false;
serialization_interval_sec = 5;


## Example Output

The key complexity lies in meticulous low-level packet manipulation, combined with concurrency control, timing, and simulation of complex network behaviors (out-of-order delivery, delayed sending, etc.).

Given most of the informaiton is done in post-processing, several JSON files are currently included with configurable options. Here is the sample output for a throughput of the information: